# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

### Load Spark and Data

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('clusterprojectebc').getOrCreate()

In [3]:
# Load data
data = spark.read.csv("hack_data.csv",header=True,inferSchema=True)

In [4]:
data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [6]:
data.limit(5).toPandas()

,Session_Connection_Time,Bytes Transferred,Kali_Trace_Used,Servers_Corrupted,Pages_Corrupted,Location,WPM_Typing_Speed
0,8.0,391.09,1,2.96,7.0,Slovenia,72.37
1,20.0,720.99,0,3.04,9.0,British Virgin Islands,69.08
2,31.0,356.32,1,3.71,8.0,Tokelau,70.58
3,2.0,228.08,1,2.48,8.0,Bolivia,70.80
4,20.0,408.50,0,3.57,8.0,Iraq,71.28


In [10]:
data.describe().toPandas()

,summary,Session_Connection_Time,Bytes Transferred,Kali_Trace_Used,Servers_Corrupted,Pages_Corrupted,Location,WPM_Typing_Speed
0,count,334,334,334,334,334,334,334
1,mean,30.008982035928145,607.2452694610777,0.5119760479041916,5.258502994011977,10.838323353293413,None,57.342395209580864
2,stddev,14.088200614636158,286.33593163576757,0.5006065264451406,2.30190693339697,3.06352633036022,None,13.41106336843464
3,min,1.0,10.0,0,1.0,6.0,Afghanistan,40.0
4,max,60.0,1330.5,1,10.0,15.0,Zimbabwe,75.0


### Clean Data

In [11]:
# list number of NANs or NULLs in each column
from pyspark.sql.functions import count, when, isnan, col
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).toPandas()

,Session_Connection_Time,Bytes Transferred,Kali_Trace_Used,Servers_Corrupted,Pages_Corrupted,Location,WPM_Typing_Speed
0,0,0,0,0,0,0,0


In [15]:
cleaned_data=data

### Prepare data

In [29]:
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, StandardScaler, VectorAssembler
from pyspark.ml import Pipeline

#### One-hop encode 'location'

In [109]:
# list all string columns where we will apply the transformations
numerical_columns = ['Session_Connection_Time','Bytes Transferred','Kali_Trace_Used',
                     'Servers_Corrupted','Pages_Corrupted','WPM_Typing_Speed']
categorical_columns = ['Location']

# create list with stages that will compose the pipeline
pipeline_stages = [] 

In [110]:
# add the transformations on categorical columns as stages in the Pipeline
for col in categorical_columns:
    # Category Indexing with StringIndexer
    stringIndexer = StringIndexer(inputCol=col, outputCol=col+"Index")

    # Use OneHotEncoder to convert categorical variables into binary SparseVectors
    oneHotEncoder = OneHotEncoderEstimator(inputCols=[col+"Index"], outputCols=[col+"Vec"])

    # Add into pipeline stages
    pipeline_stages += [stringIndexer, oneHotEncoder]

#### Create vector features

In [111]:
# Create a list of all the features to be assembled into a vector
#assembler_cols = list(map(lambda c: c + "Vec", categorical_columns)) + numerical_columns
assembler_cols = numerical_columns

# Create the features vector
vectorAssembler = VectorAssembler(inputCols = assembler_cols, outputCol='features')

# Add into pipeline stages
pipeline_stages += [vectorAssembler]

#### Scale features

In [112]:
# create the scaler to scale the features
featureScaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

# Add into pipeline stages
pipeline_stages += [featureScaler]

In [113]:
pipeline_stages

[StringIndexer_43e5ba3b455154bc29bb,
 OneHotEncoderEstimator_4bbc863eb604678b8e5c,
 VectorAssembler_40768bcfd989d7b97919,
 StandardScaler_438a8606619a3163469c]

#### Manually test pipeline stages

In [114]:
#data1 = pipeline_stages[0].fit(cleaned_data.limit(5)).transform(cleaned_data.limit(5))
#data1.toPandas()

In [115]:
#data2 = pipeline_stages[1].fit(data1).transform(data1)
#data2.toPandas()['LocationVec']

In [116]:
#data3 = pipeline_stages[2].transform(data2)
#data3.toPandas()['features']

In [117]:
#data4 = pipeline_stages[3].fit(data3).transform(data3)
#data4.toPandas()['scaledFeatures']

#### Create and Run pipeline

In [118]:
# Create a Pipeline with all previous actions
features_pipeline = Pipeline(stages=pipeline_stages)

In [95]:
# check against the manually executed stages
#features_pipeline.fit(cleaned_data.limit(5)).transform(cleaned_data.limit(5)).select('scaledFeatures').toPandas()

In [124]:
# Run the feature transformations
final_data = features_pipeline.fit(cleaned_data).transform(cleaned_data)

In [126]:
final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- LocationIndex: double (nullable = false)
 |-- LocationVec: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaledFeatures: vector (nullable = true)



## Train the clustering model and evaluate

In [127]:
from pyspark.ml.clustering import KMeans

In [163]:
# create and configure the kmeans model
kmeans = KMeans(featuresCol='scaledFeatures', k=2)

# train the model
kmeans_model = kmeans.fit(final_data)

In [161]:
# Evaluate clustering by computing "Within Set Sum of Squared Errors"
wssse = kmeans_model.computeCost(final_data)
print("WSSSE = " + str(wssse))

WSSSE = 434.1492898715845


Numerical features & Location
 - K=2 WSSSE=61599
 - K=3 WSSSE=61261
 - K=4 WSSSE=60919

Numerical features
 - K=2 WSSSE=601
 - K=3 WSSSE=434
 - K=4 WSSSE=267
 - K=5 WSSSE=400 

In [164]:
kmeans_model.transform(final_data).select('prediction').groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



## 2 hackers!

In [166]:
# 3 hackers
#+----------+-----+
#|prediction|count|
#+----------+-----+
#|         1|   83|
#|         2|   84|
#|         0|  167|
#+----------+-----+

# 2 hackers
#+----------+-----+
#|prediction|count|
#+----------+-----+
#|         1|  167|
#|         0|  167|
#+----------+-----+